In [1]:
import pandas as pd               # Pandas handles dataframes
import numpy as np                # Numpy handles lots of basic maths operations
import matplotlib.pyplot as plt   # Matplotlib for plotting
import seaborn as sns             # Seaborn for beautiful plots
from dateutil import *            # I prefer dateutil for parsing dates
import math                       # transformations
import statsmodels.formula.api as smf  # for doing statistical regression
import statsmodels.api as sm      # access to the wider statsmodels library, including R datasets
from collections import Counter   # Counter is useful for grouping and counting
import scipy

In [2]:
df = pd.read_stata("data/close_college.dta")
display(df.head())
df.shape

,nearc4,educ,black,smsa,south,married,exper,lwage
0,0,7,1,1,0,1.0,16,6.306275
1,0,12,0,1,0,1.0,9,6.175867
2,0,12,0,1,0,1.0,16,6.580639
3,1,11,0,1,0,1.0,10,5.521461
4,1,12,0,1,0,1.0,16,6.591674


(3010, 8)

In [3]:
data = df

In [4]:

# Define your independent variables (features)
X = data[['educ', 'black', 'exper', 'smsa']]

# Add a constant for the intercept term
X = sm.add_constant(X)

# Define the dependent variable
y = data['lwage']

# Fit the regression model
model = sm.OLS(y, X).fit()

# Print the summary of the regression results
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.262
Model:                            OLS   Adj. R-squared:                  0.261
Method:                 Least Squares   F-statistic:                     266.0
Date:                Fri, 15 Mar 2024   Prob (F-statistic):          5.96e-196
Time:                        11:33:03   Log-Likelihood:                -1369.0
No. Observations:                3010   AIC:                             2748.
Df Residuals:                    3005   BIC:                             2778.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.8225      0.063     76.628      0.0

## Simple model: 

for the first stage I want to regress nearc4 on educ

for the second stage I want to regress educ on lwage

In [17]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Assuming you have your data loaded into a DataFrame called 'df'

# First stage regression: nearc4 on educ
X_first_stage = df[['educ']]
X_first_stage = sm.add_constant(X_first_stage)  # Adding constant for the intercept
y_first_stage = df['nearc4']

first_stage_model = sm.OLS(y_first_stage, X_first_stage)
first_stage_results = first_stage_model.fit()

# Print first stage results
print(first_stage_results.summary())

# Extract the predicted values of 'nearc4'
df['predicted_nearc4'] = first_stage_results.predict(X_first_stage)

# Second stage regression: lwage on educ (using predicted_nearc4 as instrument)
X_second_stage = df[['educ', 'predicted_nearc4']]
X_second_stage = sm.add_constant(X_second_stage)  # Adding constant for the intercept
y_second_stage = df['lwage']

second_stage_model = sm.OLS(y_second_stage, X_second_stage)
second_stage_results = second_stage_model.fit()

# Print second stage results
print(second_stage_results.summary())


                            OLS Regression Results                            
Dep. Variable:                 nearc4   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     63.91
Date:                Fri, 15 Mar 2024   Prob (F-statistic):           1.84e-15
Time:                        11:59:11   Log-Likelihood:                -1938.9
No. Observations:                3010   AIC:                             3882.
Df Residuals:                    3008   BIC:                             3894.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3492      0.042      8.221      0.0

# Er gaat ws iets mis met multicollunarity

In [18]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculate VIF for the exogenous variables
vif = pd.DataFrame()
vif["Variable"] = df[['educ', 'black', 'exper', 'smsa']].columns
vif["VIF"] = [variance_inflation_factor(df[['educ', 'black', 'exper', 'smsa']].values, i) for i in range(len(df[['educ', 'black', 'exper', 'smsa']].columns))]

print(vif)

  Variable       VIF
0     educ -0.000873
1    black -0.121936
2    exper -0.000629
3     smsa  0.163834


In [20]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Assuming you have your data loaded into a DataFrame called 'df'
# And 'nearc4' is the endogenous variable, 'educ' is the exogenous variable, and 'lwage' is the outcome variable

# First stage regression: nearc4 on educ
X_first_stage = df[['educ']]
X_first_stage = sm.add_constant(X_first_stage)  # Adding constant for the intercept
y_first_stage = df['nearc4']

# Estimate the first stage regression
first_stage_model = sm.OLS(y_first_stage, X_first_stage)
first_stage_results = first_stage_model.fit()

# Extract fitted values from the first stage
df['predicted_nearc4'] = first_stage_results.fittedvalues

# Second stage regression: lwage on predicted_nearc4
X_second_stage = df[['predicted_nearc4']]
X_second_stage = sm.add_constant(X_second_stage)  # Adding constant for the intercept
y_second_stage = df['lwage']

# Estimate the second stage regression
second_stage_model = sm.OLS(y_second_stage, X_second_stage)
second_stage_results = second_stage_model.fit()

# Print summary of second stage results
print(second_stage_results.summary())



                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.099
Model:                            OLS   Adj. R-squared:                  0.098
Method:                 Least Squares   F-statistic:                     329.5
Date:                Fri, 15 Mar 2024   Prob (F-statistic):           5.77e-70
Time:                        12:03:54   Log-Likelihood:                -1668.8
No. Observations:                3010   AIC:                             3342.
Df Residuals:                    3008   BIC:                             3354.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                4.8460      0.078  

# Meerdere variabelen toegevoegd:

for the first stage I want to regress nearc4 on educ

for the second stage I want to predict lwage using the first stage and black, exper, sma

In [21]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Assuming you have your data loaded into a DataFrame called 'df'

# First stage regression: nearc4 on educ
X_first_stage = df[['educ']]
X_first_stage = sm.add_constant(X_first_stage)  # Adding constant for the intercept
y_first_stage = df['nearc4']

first_stage_model = sm.OLS(y_first_stage, X_first_stage)
first_stage_results = first_stage_model.fit()

# Extract the predicted values of 'nearc4'
df['predicted_nearc4'] = first_stage_results.predict(X_first_stage)

# Second stage regression: lwage on predicted_nearc4, black, exper, smsa
X_second_stage = df[['predicted_nearc4', 'black', 'exper', 'smsa']]
X_second_stage = sm.add_constant(X_second_stage)  # Adding constant for the intercept
y_second_stage = df['lwage']

second_stage_model = sm.OLS(y_second_stage, X_second_stage)
second_stage_results = second_stage_model.fit()

# Print summary of second stage results
print(second_stage_results.summary())


                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.262
Model:                            OLS   Adj. R-squared:                  0.261
Method:                 Least Squares   F-statistic:                     266.0
Date:                Fri, 15 Mar 2024   Prob (F-statistic):          5.96e-196
Time:                        12:05:35   Log-Likelihood:                -1369.0
No. Observations:                3010   AIC:                             2748.
Df Residuals:                    3005   BIC:                             2778.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                3.7626      0.111  

# Meerdere IV


Zijn hier de IV's goed, moet eht niet andersom?

In [22]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Assuming you have your data loaded into a DataFrame called 'df'

# First stage regression: nearc4 on educ, and south on black
X_first_stage = df[['educ', 'black', 'south']]
X_first_stage = sm.add_constant(X_first_stage)  # Adding constant for the intercept
y_nearc4 = df['nearc4']
y_south = df['south']

# Estimating first stage regressions
first_stage_model_nearc4 = sm.OLS(y_nearc4, X_first_stage)
first_stage_results_nearc4 = first_stage_model_nearc4.fit()

first_stage_model_south = sm.OLS(y_south, X_first_stage)
first_stage_results_south = first_stage_model_south.fit()

# Extracting predicted values from the first stage
df['predicted_nearc4'] = first_stage_results_nearc4.predict(X_first_stage)
df['predicted_south'] = first_stage_results_south.predict(X_first_stage)

# Second stage regression: lwage on predicted_nearc4 and predicted_south
X_second_stage = df[['predicted_nearc4', 'predicted_south']]
X_second_stage = sm.add_constant(X_second_stage)  # Adding constant for the intercept
y_second_stage = df['lwage']

# Estimating second stage regression
second_stage_model = sm.OLS(y_second_stage, X_second_stage)
second_stage_results = second_stage_model.fit()

# Print summary of second stage results
print(second_stage_results.summary())


                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.128
Model:                            OLS   Adj. R-squared:                  0.127
Method:                 Least Squares   F-statistic:                     220.8
Date:                Fri, 15 Mar 2024   Prob (F-statistic):           3.35e-90
Time:                        12:10:04   Log-Likelihood:                -1619.0
No. Observations:                3010   AIC:                             3244.
Df Residuals:                    3007   BIC:                             3262.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                4.8360      0.120  

# Nu met 2 regressies



    Two separate first-stage regressions are performed: one regressing educ on nearc4 and the other regressing black on south.
    The predicted values of educ and black are extracted from the first-stage regressions.
    These predicted values are then used in the second-stage regression to predict lwage.

In [25]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Assuming you have your data loaded into a DataFrame called 'df'

# First stage regression: educ on nearc4
X_first_stage_educ = df[['nearc4']]
X_first_stage_educ = sm.add_constant(X_first_stage_educ)  # Adding constant for the intercept
y_first_stage_educ = df['educ']

first_stage_model_educ = sm.OLS(y_first_stage_educ, X_first_stage_educ)
first_stage_results_educ = first_stage_model_educ.fit()

# Extracting predicted values from the first stage for educ
df['predicted_educ'] = first_stage_results_educ.predict(X_first_stage_educ)

# First stage regression: black on south
X_first_stage_black = df[['south']]
X_first_stage_black = sm.add_constant(X_first_stage_black)  # Adding constant for the intercept
y_first_stage_black = df['black']

first_stage_model_black = sm.OLS(y_first_stage_black, X_first_stage_black)
first_stage_results_black = first_stage_model_black.fit()

# Extracting predicted values from the first stage for black
df['predicted_black'] = first_stage_results_black.predict(X_first_stage_black)

# Second stage regression: lwage on predicted_educ and predicted_black
X_second_stage = df[['predicted_educ', 'predicted_black']]
X_second_stage = sm.add_constant(X_second_stage)  # Adding constant for the intercept
y_second_stage = df['lwage']

second_stage_model = sm.OLS(y_second_stage, X_second_stage)
second_stage_results = second_stage_model.fit()

# Print summary of second stage results
print(second_stage_results.summary())


                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.090
Method:                 Least Squares   F-statistic:                     150.7
Date:                Fri, 15 Mar 2024   Prob (F-statistic):           4.26e-63
Time:                        12:18:29   Log-Likelihood:                -1681.5
No. Observations:                3010   AIC:                             3369.
Df Residuals:                    3007   BIC:                             3387.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               4.8347      0.275     

# Now including everything


Als het goed is klopt deze helemaal met 2 IV dingen :


In [27]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Assuming you have your data loaded into a DataFrame called 'df'

# First stage regression: educ on nearc4
X_first_stage_educ = df[['nearc4']]
X_first_stage_educ = sm.add_constant(X_first_stage_educ)  # Adding constant for the intercept
y_first_stage_educ = df['educ']

first_stage_model_educ = sm.OLS(y_first_stage_educ, X_first_stage_educ)
first_stage_results_educ = first_stage_model_educ.fit()

# Extracting predicted values from the first stage for educ
df['predicted_educ'] = first_stage_results_educ.predict(X_first_stage_educ)

# First stage regression: black on south
X_first_stage_black = df[['south']]
X_first_stage_black = sm.add_constant(X_first_stage_black)  # Adding constant for the intercept
y_first_stage_black = df['black']

first_stage_model_black = sm.OLS(y_first_stage_black, X_first_stage_black)
first_stage_results_black = first_stage_model_black.fit()

# Extracting predicted values from the first stage for black
df['predicted_black'] = first_stage_results_black.predict(X_first_stage_black)

# Second stage regression: lwage on predicted_educ and predicted_black
X_second_stage = df[['predicted_educ', 'predicted_black', 'smsa', 'exper']]
X_second_stage = sm.add_constant(X_second_stage)  # Adding constant for the intercept
y_second_stage = df['lwage']

second_stage_model = sm.OLS(y_second_stage, X_second_stage)
second_stage_results = second_stage_model.fit()

# Print summary of second stage results
print(second_stage_results.summary())


                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.121
Model:                            OLS   Adj. R-squared:                  0.120
Method:                 Least Squares   F-statistic:                     103.7
Date:                Fri, 15 Mar 2024   Prob (F-statistic):           7.21e-83
Time:                        12:21:13   Log-Likelihood:                -1630.5
No. Observations:                3010   AIC:                             3271.
Df Residuals:                    3005   BIC:                             3301.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               5.5019      0.282     